In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import warnings
import pingouin as pg
import kdephys as kde

from acr.utils import PAPER_FIGURE_ROOT

warnings.filterwarnings('ignore')

plt.style.use('fast')
plt.style.use('/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle')
#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
from pub_utils import *
data_agg = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from data_agg import *

In [ ]:
sub = 'ACR_26'
exp = 'swi'

# Hypnogram

In [ ]:
h = acr.io.load_hypno_full_exp(sub, exp)
sd_start, stim_start, stim_end, reb_start, full_x_start = acr.info_pipeline.get_sd_exp_landmarks(sub, exp, update=False, return_early=False)

hd = acr.hypnogram_utils.create_acr_hyp_dict(sub, exp)


bl_day = str(full_x_start).split(' ')[0]
fake_x_start = pd.to_datetime(bl_day + ' 09:00:00')
wake_states = ['Wake-Good', 'Transition-to-Wake', 'Sort-Exclude', 'Transition-to-NREM', 'Brief-Arousal', 'Art']
REM_states = ['REM', 'Transition-to-REM']

h.loc[h['state'].isin(wake_states), 'state'] = 'Wake'
h.loc[h['state'].isin(REM_states), 'state'] = 'REM'


# plotting adjustments
h.loc[((h['end_time'] > sd_start) & (h['end_time'] < stim_end)), 'state'] = 'Wake'

reb_end = hd['rebound'].end_time.max()
h = acr.hypnogram_utils.standard_hypno_corrections(h)

# Simple Colored Hypnogram

In [ ]:
h2p = h.trim_select(sd_start, reb_end)

In [ ]:
f, ax = kde.plot.main.plot_basic_hypnogram(h2p, style_path='/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle', single_tone=True)

ax.axvline(stim_start, color='black', linestyle='--', linewidth=3)
ax.axvline(stim_end, color='black', linestyle='--', linewidth=3)

f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_hypnogram.png', dpi=600)

In [ ]:
f, ax = kde.plot.main.plot_basic_hypnogram(h2p, style_path='/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle', single_tone=True)

f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_hypnogram--NO_LINES.png', dpi=600)

In [ ]:
f, ax = kde.plot.main.plot_basic_hypnogram(h2p, style_path='/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle', single_tone=True)

sb1 = sd_start 
sb2 = sd_start+pd.Timedelta('1h')
# Draw a horizontal scale bar from sb1 to sb2 (1 hour duration)
ax.plot([sb1, sb2], [0.1, 0.1], color='black', linewidth=4)  # Position the scale bar at y=0.1

f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_hypnogram--NO_LINES--ScaleBar.png', dpi=600)

In [ ]:
full_duration = (reb_end - sd_start).total_seconds()
pix_per_hr = (1600/full_duration)*3600
print(f'{pix_per_hr:.2f} pixels per hour')
for hr in [1, 2, 3, 4, 5, 6]:
    print(f'{hr*pix_per_hr:.2f} pixels')


## Full 48 hr Hypnogram

In [ ]:
endt = fake_x_start + pd.Timedelta('48h')
h48 = h.trim_select(fake_x_start, endt)

f, ax = kde.plot.main.plot_basic_hypnogram(h48, style_path='/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle', xlim=(fake_x_start, endt), single_tone=True)

#f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_hypnogram--48HR.png', dpi=600)

# EMG Plot

In [ ]:
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['xtick.bottom'] = False

In [ ]:
recs = acr.info_pipeline.get_exp_recs(sub, exp)
emg = acr.io.load_concat_raw_data(sub, recs, stores=['EMGr'])
e2p = emg.ts(sd_start, reb_end).ch(1).sel(store='EMGr')
emg48 = emg.ts(fake_x_start, endt).ch(1).sel(store='EMGr')

In [ ]:
f, ax = plt.subplots(figsize=(20, 0.4))
ax.set_xlim(sd_start, reb_end)
ax.set_yticks([])
ax.plot(e2p.datetime, e2p.data, color='#2b3e4b')
ax.set_ylim(-2000, 3000)
f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_emg.png', dpi=600, bbox_inches='tight', transparent=True, facecolor='none')

In [ ]:
f, ax = plt.subplots(figsize=(20, 0.4))
ax.set_xlim(fake_x_start, endt)
ax.set_yticks([])
ax.plot(emg48.datetime, emg48.data, color=EMG_SLATE)
ax.set_ylim(-2000, 3000)
f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_emg--48HR.png', dpi=600, bbox_inches='tight', transparent=True, facecolor='none')

In [ ]:
f, ax = plt.subplots(figsize=(20, 0.4))
ax.set_xlim(sd_start, reb_end)
ax.set_yticks([])
ax.plot(e2p.datetime, e2p.data, color=EMG_SLATE)
ax.set_ylim(-2000, 3000)
ax.axhspan(-2000, 3000, xmin=0.8, xmax=0.95, color='red', alpha=0.5)
f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_emg--5000mv.png', dpi=600, bbox_inches='tight', transparent=True, facecolor='none')


# SWA Plot

In [ ]:
chan_to_plot = 10

In [ ]:
_bp = load_raw_bp_df(sub, exp)
bp = make_raw_bp_df_relative_to_baseline(_bp, col_to_use='full_bl', value_to_use='True', state_to_use='NREM', method='mean')

In [ ]:
# Filter out a single channel to use
swa = bp.filter(pl.col('band') == 'delta')
swa_plot = swa.ts(sd_start, reb_end)
swa_plot = swa_plot.to_pandas()
swa_plot_o = swa_plot.prb('NNXo').chnl(chan_to_plot)
swa_plot_r = swa_plot.prb('NNXr').chnl(chan_to_plot)
# set the clearly arifactual periods to 0
swa_plot_r.loc[((swa_plot_r['datetime']<reb_start) & (swa_plot_r['bandpower_rel']>0.8)), 'bandpower_rel'] = 0
swa_plot_o.loc[((swa_plot_o['datetime']<stim_start) & (swa_plot_o['bandpower_rel']>0.8)), 'bandpower_rel'] = 0

In [ ]:
# Plot Both Probes
f, ax = plt.subplots(figsize=(25, 2.5))
ax.set_xlim(sd_start, reb_end)
ax.set_yticks([])
ax.plot(swa_plot_o.datetime, swa_plot_o.sm('bandpower_rel', sigma=6)['bandpower_rel'], color=SOM_BLUE, linewidth=3, alpha=1)
ax.plot(swa_plot_r.datetime, swa_plot_r.sm('bandpower_rel', sigma=6)['bandpower_rel'], color=NNXR_GRAY, linewidth=3, alpha=0.8)
ax.set_ylim(0, 2.5)
f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_NNXo_delta.png', dpi=600, bbox_inches='tight', transparent=True, facecolor='none')

In [ ]:
# Plot Both Probes - with Yscale!
f, ax = plt.subplots(figsize=(25, 2.5))
ax.set_xlim(sd_start, reb_end)
ax.set_yticks([])
ax.plot(swa_plot_o.datetime, swa_plot_o.sm('bandpower_rel', sigma=6)['bandpower_rel'], color=SOM_BLUE, linewidth=3, alpha=1)
ax.plot(swa_plot_r.datetime, swa_plot_r.sm('bandpower_rel', sigma=6)['bandpower_rel'], color=NNXR_GRAY, linewidth=3, alpha=0.8)
ax.set_ylim(0, 2.5)
ax.axhspan(0, 2, xmin=0.3, xmax=0.32, color='red', alpha=0.7)
ax.axhline(1, color='green', linestyle='-', linewidth=3)
f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_NNXo_delta--YSCALe__0-2PctofBL.png', dpi=600, bbox_inches='tight', transparent=True, facecolor='none')

In [ ]:
swa_plot = swa.ts(stim_start-pd.Timedelta('2m'), reb_end)
swa_plot = swa_plot.to_pandas()
swa_plot_o = swa_plot.prb('NNXo').chnl(chan_to_plot)
swa_plot_r = swa_plot.prb('NNXr').chnl(chan_to_plot)

# set the clearly arifactual periods to 0
swa_plot_r.loc[((swa_plot_r['datetime']<reb_start) & (swa_plot_r['bandpower_rel']>0.8)), 'bandpower_rel'] = 0
swa_plot_o.loc[((swa_plot_o['datetime']<stim_start) & (swa_plot_o['bandpower_rel']>0.8)), 'bandpower_rel'] = 0

In [ ]:
# Plot Both Probes - with Yscale!
f, ax = plt.subplots(figsize=(30, 6))
#ax.set_xlim(sd_start, reb_end)
ax.set_yticks([])
ax.plot(swa_plot_o.datetime, swa_plot_o.sm('bandpower_rel', sigma=3)['bandpower_rel'], color=SOM_BLUE, linewidth=3, alpha=1)
ax.plot(swa_plot_r.datetime, swa_plot_r.sm('bandpower_rel', sigma=3)['bandpower_rel'], color=NNXR_GRAY, linewidth=3, alpha=0.8)
ax.set_ylim(0, 3)
ax.axhspan(0, 2, xmin=0.3, xmax=0.32, color='red', alpha=0.7)
ax.axhline(1, color='green', linestyle='-', linewidth=3)
f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_NNXo_delta--ZOOMED_SCALED.png', dpi=600, bbox_inches='tight', transparent=True, facecolor='none')